In [1]:
###### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-prompt-recovery/sample_submission.csv
/kaggle/input/llm-prompt-recovery/train.csv
/kaggle/input/llm-prompt-recovery/test.csv
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/llm-prompt-recovery-synthetic-datastore/gemma1000_w7b.csv
/kaggle/input/3000-rewritten-texts-prompt-recovery-challenge/prompts_0_500_wiki_first_para_3000.csv
/kaggle/input/3000-rewritten-texts-prompt-recovery-challenge/rewrite_prompts.csv


In [2]:
Train=pd.read_csv('/kaggle/input/llm-prompt-recovery/train.csv')
test=pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')
sb=pd.read_csv('/kaggle/input/llm-prompt-recovery/sample_submission.csv')

### Exploring Dataset

In [3]:
Train

,id,original_text,rewrite_prompt,rewritten_text
0,-1,The competition dataset comprises text passage...,"Convert this into a sea shanty: """"""The competi...",Here is your shanty: (Verse 1) The text is rew...


In [4]:
Train.original_text[0]

'The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.'

In [5]:
Train.rewrite_prompt[0]

'Convert this into a sea shanty: """The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set."""'

In [6]:
Train.rewritten_text[0]

"Here is your shanty: (Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be"

In [7]:
test

,id,original_text,rewritten_text
0,-1,The competition dataset comprises text passage...,Here is your shanty: (Verse 1) The text is rew...


In [8]:
sb

,id,rewrite_prompt
0,9559194,Improve that text.


In [9]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown

In [10]:
class CFG:
    seed = 42
    dataset_path = "/kaggle/input/llm-prompt-recovery"
    preset = "gemma_instruct_2b_en" # name of pretrained Gemma
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    epochs = 1 # number of epochs to train

In [11]:
keras.utils.set_random_seed(CFG.seed)

In [12]:
# `LLM Prompt Recovery - Synthetic Datastore dataset` by @dschettler8845
df1 = pd.read_csv("/kaggle/input/llm-prompt-recovery-synthetic-datastore/gemma1000_w7b.csv")
df1 = df1[["original_text", "rewrite_prompt", "gemma_7b_rewritten_text_temp0"]]
df1 = df1.rename(columns={"gemma_7b_rewritten_text_temp0":"rewritten_text"})
df1.head(2)

# `3000 Rewritten texts - Prompt recovery Challenge` by @dipamc77
df2 = pd.read_csv("/kaggle/input/3000-rewritten-texts-prompt-recovery-challenge/prompts_0_500_wiki_first_para_3000.csv")
df2.head(2)

# Merge all datasets
df = pd.concat([df1, df2], axis=0)
df = df.sample(2000).reset_index(drop=True) # to reduce training time we are only using 2k samples
df.head(5)

,original_text,rewrite_prompt,rewritten_text
0,Story highlights New findings suggest sperm al...,Turn this into a programmer's code.,```python\n# Code to summarize the text\n\nspe...
1,Glasgow's Needy is a non-registered charity or...,Make this a formal apology letter to a customer.,"[Your Name]\n[Your Address]\nGlasgow, Scotland..."
2,"Pune: In a bizarre case, a parrot accused of ""...",Convert this into a technique to be used.,**Technique:**\n\n**Step 1: Gather evidence.**...
3,Berrya is a genus of evergreen trees with fibr...,Convert the text into the layout of a classic ...,## Level Layout: Berrya Forest\n\n**Background...
4,The Libertarian Party of Oregon is a political...,Recast it as the backstory for a mysterious an...,In the shadows of the Evergreen State of Orego...


In [13]:
df.shape

(2000, 3)

In [14]:
template="""Instruction:\n Below are the 'Original Text' that have been re-written/changed/improve to 'rewritten text' by using different prompts using 'Gemma 7b-it' so your task is carefully read the pattern between 'Orignal text' and 'rewritten text' and identify the prompts that was likely used to convert Original text to these rewritten/improved texts.\n\nOriginal Text: \n{original_text}\n\nRewritten Text : \n{rewritten_text}\n\nResponse:\n{rewrite_prompt}"""


In [15]:
template = """Instruction:\nBelow, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 7b-it` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.\n\nOriginal Text:\n{original_text}\n\nRewriten Text:\n{rewritten_text}\n\nResponse:\n{rewrite_prompt}"""

In [16]:
df

,original_text,rewrite_prompt,rewritten_text
0,Story highlights New findings suggest sperm al...,Turn this into a programmer's code.,```python\n# Code to summarize the text\n\nspe...
1,Glasgow's Needy is a non-registered charity or...,Make this a formal apology letter to a customer.,"[Your Name]\n[Your Address]\nGlasgow, Scotland..."
2,"Pune: In a bizarre case, a parrot accused of ""...",Convert this into a technique to be used.,**Technique:**\n\n**Step 1: Gather evidence.**...
3,Berrya is a genus of evergreen trees with fibr...,Convert the text into the layout of a classic ...,## Level Layout: Berrya Forest\n\n**Background...
4,The Libertarian Party of Oregon is a political...,Recast it as the backstory for a mysterious an...,In the shadows of the Evergreen State of Orego...
...,...,...,...
1995,Coding Contest Byte: The Square Root Trick Cos...,Frame this as a gardener's journal.,## The Square Root Trick - A Gardener's Journa...
1996,Political types aren't the only ones descendin...,Rewrite this as a seminar to be hosted.,**Seminar Title:** Unlocking Economic Growth T...
1997,Venom is the fifth studio album by Welsh heavy...,Change it into an old-school hip-hop song.,"(Verse 1)\nYo, listen up cause I'm about to dr..."
1998,Sofia the First is an American computer-animat...,Rewrite it as a classic noir film voice-over.,(The voice dripping like a wet cigarette smoke...


In [17]:
df['prompt']=df.progress_apply(lambda row: template.format(original_text=row.original_text,rewritten_text=row.rewritten_text,rewrite_prompt=row.rewrite_prompt),axis=1)
                        

  0%|          | 0/2000 [00:00<?, ?it/s]

In [18]:
df.prompt

0       Instruction:\nBelow, the `Original Text` passa...
1       Instruction:\nBelow, the `Original Text` passa...
2       Instruction:\nBelow, the `Original Text` passa...
3       Instruction:\nBelow, the `Original Text` passa...
4       Instruction:\nBelow, the `Original Text` passa...
                              ...                        
1995    Instruction:\nBelow, the `Original Text` passa...
1996    Instruction:\nBelow, the `Original Text` passa...
1997    Instruction:\nBelow, the `Original Text` passa...
1998    Instruction:\nBelow, the `Original Text` passa...
1999    Instruction:\nBelow, the `Original Text` passa...
Name: prompt, Length: 2000, dtype: object

In [19]:
def colorize_text(text):
    for word,color in zip(['Instruction','Original Text','Rewritten Text','Response'],
                         ['Red','Yellow','orange','Green']):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text    

In [20]:
data=df.prompt.to_list()

In [21]:
sample=data[10]

In [22]:
change=colorize_text(sample)

In [23]:
display(Markdown(change))



**<font color='Red'>Instruction:</font>**
Below, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 7b-it` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.



**<font color='Yellow'>Original Text:</font>**
Zoltán Tóth (born 24 August 1979) is a Hungarian former competitive figure skater. He is a five-time Hungarian national champion and competed in two Winter Olympics.

Rewriten Text:
Sure, here's the rewritten text as if it were a list of ingredients in a magical potion:

The ingredients to create the potion:

Zoltán Tóth's birth date: 24 August 1979
Hungarian heritage
Five-time Hungarian national champion
Two Winter Olympics participation



**<font color='Green'>Response:</font>**
Rewrite the text as if it were a list of ingredients in a magical potion.

In [24]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")
#gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")
gemma_lm

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


<GemmaCausalLM name=gemma_causal_lm, built=True>

In [25]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

!pip install -U keras-nlp
!pip install -U keras

!pip install -q -U keras>=3

import kagglehub

# Download latest version
path = kagglehub.model_download("keras/gemma/keras/gemma_instruct_2b_en")

print("Path to model files:", path)

In [26]:
x, y, sample_weight = gemma_lm.preprocessor(data[0:2])

In [27]:
x

{'token_ids': Array([[     2,  37854, 235292, ...,      0,      0,      0],
        [     2,  37854, 235292, ...,      0,      0,      0]],      dtype=int32),
 'padding_mask': Array([[ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False]], dtype=bool)}

In [28]:
x['token_ids'].shape

(2, 1024)

In [29]:
y.shape

(2, 1024)

In [30]:
sample_weight.shape

(2, 1024)

### Testing before fine tuning

In [31]:
row=df.iloc[15]
prompt=template.format(
       original_text=row.original_text,
       rewritten_text=row.rewritten_text,
       rewrite_prompt='')

output=gemma_lm.generate(prompt,max_length=512)
output=colorize_text(output)

In [32]:
display(Markdown(output))



**<font color='Red'>Instruction:</font>**
Below, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 7b-it` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.



**<font color='Yellow'>Original Text:</font>**
Esports were featured at the 2018 Asian Games as a demonstration sport, meaning medals won in this sport would not be counted in the official overall medal tally. It was held from 26 August to 1 September 2018. Six video games were featured in the demonstration event, Arena of Valor, Clash Royale, Hearthstone, League of Legends, Pro Evolution Soccer, and StarCraft II.

Rewriten Text:
In the heart of the celestial inferno, where scorching flames danced and cosmic embers ignited the sky, a civilization bloomed beneath the blazing sun. Known as Stellaris, they carved their existence within the heart of a celestial furnace, a testament to the enduring spirit of humanity.

Stellaris had harnessed the radiant energy of the star, harnessing its scorching power to fuel their advanced technology and illuminate their home. Their cities pulsed with the rhythm of celestial waves, radiant towers piercing through the aether, a reflection of their ambition and resilience.

Their lives revolved around the enigmatic star, its luminescence casting long shadows upon the celestial canvas below.



**<font color='Green'>Response:</font>**
The specific prompt or instruction that was likely given to the LLM to rewrite/transform the text in this way is not explicitly mentioned in the context, so I cannot infer the information from the context.

### Fine Tuning with Lora

In [33]:
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [34]:
gemma_lm.preprocessor.sequence_length= CFG.sequence_length

gemma_lm.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=keras.optimizers.Adam(learning_rate=3e-5),
                weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [35]:
gemma_lm.fit(data,epochs=CFG.epochs,batch_size=CFG.batch_size)

2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1479s 730ms/step - loss: 1.3144 - sparse_categorical_accuracy: 0.6193


### Test Data

In [36]:
test_df=pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')

In [37]:
test_df

,id,original_text,rewritten_text
0,-1,The competition dataset comprises text passage...,Here is your shanty: (Verse 1) The text is rew...


In [38]:
prompt=template.format(original_text=test_df.original_text,
                      rewritten_text=test_df.rewritten_text,
                      rewrite_prompt='')
output=gemma_lm.generate(prompt,max_length=512)

In [39]:
output=colorize_text(output)
display(Markdown(output))



**<font color='Red'>Instruction:</font>**
Below, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 7b-it` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.



**<font color='Yellow'>Original Text:</font>**
0    The competition dataset comprises text passage...
Name: original_text, dtype: object

Rewriten Text:
0    Here is your shanty: (Verse 1) The text is rew...
Name: rewritten_text, dtype: object



**<font color='Green'>Response:</font>**
Transform it into a shanty.

### sample_submission

In [40]:
preds = []
for i in tqdm(range(len(test_df))):
    row = test_df.iloc[i]

    # Generate Prompt using template
    prompt = template.format(
        original_text=row.original_text,
        rewritten_text=row.rewritten_text,
        rewrite_prompt=""
    )

    # Infer
    output = gemma_lm.generate(prompt, max_length=512)
    pred = output.replace(prompt, "") # remove the prompt from output
    
    # Store predictions
    preds.append([row.id, pred])

  0%|          | 0/1 [00:00<?, ?it/s]

In [41]:
sub_df = pd.DataFrame(preds, columns=["id", "rewrite_prompt"])
sub_df['rewrite_prompt'] = sub_df['rewrite_prompt'].fillna("")
sub_df['rewrite_prompt'] = sub_df['rewrite_prompt'].map(lambda x: "Improve the essay" if len(x) == 0 else x)
sub_df.to_csv("submission.csv",index=False)
sub_df.head()

,id,rewrite_prompt
0,-1,Convert this into a code competition.
